<a href="https://colab.research.google.com/github/nee23thu/Speech-emotion-recognition/blob/main/Speech_Emotion_Recognition_with_librosa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


[Dataset](https://drive.google.com/file/d/1wWsrN2Ep7x6lWqOXfr4rpKGYrJhWc8z7/view)

In [2]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
ls

drive/  sample_data/


In [4]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [5]:
import numpy as np
import librosa

def extract_feature(file_name, mfcc, chroma, mel):
    X, sample_rate = librosa.load(file_name)
    result = np.array([])
    if mfcc:
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=13).T, axis=0)
        result = np.hstack((result, mfccs))
    if chroma:
        stft = np.abs(librosa.stft(X))
        chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
        result = np.hstack((result, chroma))
    if mel:
        mel = np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T, axis=0)
        result = np.hstack((result, mel))
    return result


In [6]:
# Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

#Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [7]:
#Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("/content/drive/MyDrive/dataset/dataset/Actor_*/*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [8]:
#Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

In [9]:
x_train

array([[-5.66377441e+02,  7.43823242e+01, -1.58749952e+01, ...,
         3.97871243e-13,  3.89172500e-13,  3.82548946e-13],
       [-6.84318481e+02,  8.45214310e+01,  8.08870792e-02, ...,
         1.27088964e-12,  1.24748497e-12,  1.23265319e-12],
       [-5.98963989e+02,  8.92946854e+01, -1.97911301e+01, ...,
         2.46169645e-14,  2.31241739e-14,  2.17816115e-14],
       ...,
       [-6.15138794e+02,  6.97454224e+01, -4.61962795e+00, ...,
         2.67074665e-13,  2.60734983e-13,  2.56812990e-13],
       [-6.31096985e+02,  7.88255463e+01, -7.67340374e+00, ...,
         4.20706323e-14,  4.19644754e-14,  4.15730106e-14],
       [-6.22405457e+02,  8.72875748e+01, -2.84195976e+01, ...,
         2.46653947e-10,  2.43696396e-10,  2.41857923e-10]])

In [10]:
#Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

(384, 128)


In [11]:
#Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 153


In [12]:
#Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [13]:
#Train the model
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [14]:
#Predict for the test set
y_pred=model.predict(x_test)

In [15]:
y_pred

array(['disgust', 'calm', 'happy', 'fearful', 'fearful', 'calm',
       'disgust', 'fearful', 'happy', 'fearful', 'fearful', 'fearful',
       'fearful', 'calm', 'fearful', 'calm', 'disgust', 'disgust',
       'fearful', 'calm', 'calm', 'calm', 'happy', 'fearful', 'disgust',
       'happy', 'fearful', 'calm', 'disgust', 'calm', 'calm', 'fearful',
       'fearful', 'calm', 'happy', 'fearful', 'happy', 'fearful', 'calm',
       'calm', 'happy', 'fearful', 'disgust', 'disgust', 'calm',
       'fearful', 'calm', 'calm', 'fearful', 'disgust', 'disgust',
       'fearful', 'calm', 'happy', 'disgust', 'calm', 'calm', 'fearful',
       'calm', 'calm', 'disgust', 'fearful', 'happy', 'disgust',
       'fearful', 'calm', 'disgust', 'calm', 'disgust', 'calm', 'fearful',
       'disgust', 'disgust', 'calm', 'fearful', 'calm', 'calm', 'happy',
       'calm', 'calm', 'happy', 'fearful', 'calm', 'happy', 'disgust',
       'calm', 'fearful', 'disgust', 'calm', 'calm', 'calm', 'fearful',
       'happy', 

In [16]:
#Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

#Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 57.03%


In [17]:
from sklearn.metrics import accuracy_score, f1_score

In [18]:
f1_score(y_test, y_pred,average=None)

array([0.67532468, 0.41509434, 0.61971831, 0.50909091])

In [19]:
import pandas as pd
df=pd.DataFrame({'Actual': y_test, 'Predicted':y_pred})
df.head(20)

,Actual,Predicted
0,disgust,disgust
1,disgust,calm
2,happy,happy
3,happy,fearful
4,fearful,fearful
5,calm,calm
6,disgust,disgust
7,fearful,fearful
8,disgust,happy
9,happy,fearful


In [20]:
import pickle
# Writing different model files to file
with open( 'modelForPrediction1.sav', 'wb') as f:
    pickle.dump(model,f)

In [21]:
filename = 'modelForPrediction1.sav'
loaded_model = pickle.load(open(filename, 'rb')) # loading the model file from the storage

feature=extract_feature("/content/drive/MyDrive/dataset/dataset/Actor_01/03-01-01-01-01-01-01.wav", mfcc=True, chroma=True, mel=True)

feature=feature.reshape(1,-1)

prediction=loaded_model.predict(feature)
prediction

array(['calm'], dtype='<U7')

In [ ]:
feature

array([[-7.04823425e+02,  6.53288727e+01, -9.19415855e+00,
         2.13759346e+01, -2.19000548e-01,  6.97934771e+00,
        -8.27524948e+00,  2.18102783e-01, -1.30709763e+01,
        -2.21690679e+00,  1.03323686e+00, -4.23244810e+00,
         3.54141808e+00,  6.56251669e-01,  6.72352195e-01,
         6.52728200e-01,  6.63235545e-01,  6.83741271e-01,
         6.77614033e-01,  7.11676776e-01,  7.43286371e-01,
         7.66070426e-01,  7.46653974e-01,  7.05328822e-01,
         6.05794251e-01,  1.39239830e-06,  5.60121152e-05,
         3.62368440e-03,  2.09439714e-02,  3.13961394e-02,
         1.76842660e-02,  3.45229614e-03,  1.86057854e-02,
         2.46685669e-02,  2.04533935e-02,  2.19565798e-02,
         1.07420925e-02,  7.94340484e-03,  7.29931379e-03,
         1.51041951e-02,  3.63941453e-02,  4.98254672e-02,
         1.29398555e-02,  9.52597149e-03,  1.82860345e-02,
         3.88188213e-02,  1.36699220e-02,  2.03850423e-03,
         3.92225571e-03,  1.24087334e-02,  1.84398238e-0